In [14]:
# import pandas as pd
# import numpy as np
# #import rowgenerators as rg ****commented out for thesis****

# from sklearn.feature_selection import VarianceThreshold

# scfp = pd.read_stata('p19i6.dta')



# # SELECT only rows where there is debt and where white and black
# delinquency = 'x3005'
# loan_schedule ='x3004'# 1 on time, 5 sometimes late, 0 inapp.
# race = 'x6809'
# additional_race = 'x6810'
# hispanic = 'x7004'
# siblings = 'x5913'

# scfp = scfp[(scfp[race] == 1) | (scfp[race] == 2)]
# scfp =  scfp[(scfp[loan_schedule] == 1) | (scfp[loan_schedule] == 5)]


# # Remove revealing columns
# # race_revealing = [additional_race, 'x7004']
# # race_revealing += [col.replace('x', 'j') for col in race_revealing]
# sched_revealing = ['x7571', 'x7570', 'x7569', 'x7556', 'x7564', 'x7554','x7553',
#                    'x7534', 'x7533', 'x7532', 'x7166', 'x7529', 'x7821', 'x7844',
#                    'x7867', 'x7921', 'x7569', 'x7944', 'x7521', 'x7520', 'x7518',
#                    'x7517', delinquency, 'j3005', 'j3004']
# sched_revealing += [col.replace('x', 'j') for col in sched_revealing]

# exclude = sched_revealing#+ race_revealing
# X = scfp.copy()
# # Eliminate bad columns
# #X = scfp.loc[:, [col for col in scfp.columns if col not in exclude]]
# X = X.drop(exclude, axis=1)
# X = X.fillna(-1)

# # Eliminate columns with names starting with 'j'
# j_columns = [col for col in X.columns if col[0] == 'j']
# X = X.drop(j_columns, axis=1)

# # Remove columns with all -1

# # instantiate the VarianceThreshold object with a threshold of 0
# selector = VarianceThreshold(threshold=0)

# # fit the selector on the DataFrame
# selector.fit(X)

# # get the indices of the non-constant columns
# non_constant_cols = selector.get_support(indices=True)

# # select the non-constant columns from the original DataFrame
# X = X.iloc[:, non_constant_cols]

# # print the filtered DataFrame
# print(X.shape)



# # Change loan schedule to binary
# X[loan_schedule] = X[loan_schedule].replace({1: 0, 5: 1})
# # # only include where white or black
# # X = X[(X[race] == 1) | (X[race] == 2)]
# X[race] = X[race].replace({1: 0, 2:1})

# # Change siblings to greater less than 3
# X[siblings] = (X[siblings] > 3).astype(int)








(18362, 2494)
(18362, 2494)
(18362, 3533)


In [10]:
# Make continuous into range columns
# Define the number of bins
num_bins = 5

# Iterate through columns
for column in X.columns:
    if column == loan_schedule or column == race or column == siblings:
        continue
    if X[column].dtype != 'object' and len(X[column].unique()) > num_bins:
        # Determine bin edges
        bins = pd.qcut(X[column], q=num_bins, labels=False, duplicates='drop') # Potentially would want to do normal distruibution istead of equal bins
        X[column] = bins
    X[column] = X[column].astype('category') # Make every column categorical for now

print(X.shape)
# Get categorical columns
cat_cols = X.select_dtypes(include='category').columns
cat_one_hot = pd.get_dummies(X[cat_cols], drop_first=True)
X = X.drop(cat_cols, axis=1)
X = pd.concat([X, cat_one_hot], axis=1)
print(X.shape)

(18362, 2494)
(18362, 3531)


In [11]:
X[siblings].unique()
# find what % of people with siblings have loan_schedule == 5
print(X[X[siblings] == 1][loan_schedule].value_counts(normalize=True)) # overrepresented in loan defaults

# find what % of people without siblings have loan_schedule == 5
print(X[X[siblings] == 0][loan_schedule].value_counts(normalize=True))

x3004
0    0.811868
1    0.188132
Name: proportion, dtype: float64
x3004
0    0.866855
1    0.133145
Name: proportion, dtype: float64


In [12]:
# Split X into test and train dfs
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [13]:
# Output test and train to seperate csvs
X_train.to_csv('SCF_Siblings_train.csv', index=False)
X_test.to_csv('SCF_Siblings_test.csv', index=False)

In [53]:
import pandas as pd
import numpy as np
#import rowgenerators as rg ****commented out for thesis****

from sklearn.feature_selection import VarianceThreshold

scfp = pd.read_stata('p19i6.dta')



# SELECT only rows where there is debt and where white and black
delinquency = 'x3005'
loan_schedule ='x3004'# 1 on time, 5 sometimes late, 0 inapp.
race = 'x6809'
additional_race = 'x6810'
hispanic = 'x7004'
siblings = 'x5913'

scfp = scfp[(scfp[race] == 1) | (scfp[race] == 2)]
scfp =  scfp[(scfp[loan_schedule] == 1) | (scfp[loan_schedule] == 5)]

# Remove revealing columns
race_revealing = [additional_race, 'x7004']
race_revealing += [col.replace('x', 'j') for col in race_revealing]
sched_revealing = ['x7571', 'x7570', 'x7569', 'x7556', 'x7564', 'x7554','x7553',
                   'x7534', 'x7533', 'x7532', 'x7166', 'x7529', 'x7821', 'x7844',
                   'x7867', 'x7921', 'x7569', 'x7944', 'x7521', 'x7520', 'x7518',
                   'x7517', delinquency, 'j3005', 'j3004']
sched_revealing += [col.replace('x', 'j') for col in sched_revealing]

exclude = sched_revealing#+ race_revealing
X = scfp.copy()
# Eliminate bad columns
#X = scfp.loc[:, [col for col in scfp.columns if col not in exclude]]
X = X.drop(exclude, axis=1)
X = X.fillna(-1)

# Eliminate columns with names starting with 'j'
j_columns = [col for col in X.columns if col[0] == 'j']
X = X.drop(j_columns, axis=1)

# Remove columns with all -1

# instantiate the VarianceThreshold object with a threshold of 0
selector = VarianceThreshold(threshold=0)

# fit the selector on the DataFrame
selector.fit(X)

# get the indices of the non-constant columns
non_constant_cols = selector.get_support(indices=True)

# select the non-constant columns from the original DataFrame
X = X.iloc[:, non_constant_cols]

# print the filtered DataFrame
print(X.shape)

print(X[siblings].unique())

# Change loan schedule to binary
X[loan_schedule] = X[loan_schedule].replace({1: 0, 5: 1})
# # only include where white or black
# X = X[(X[race] == 1) | (X[race] == 2)]
X[race] = X[race].replace({1: 0, 2:1})
# Make siblings binary based on whether value is > 3
# Now, perform the operation
X[siblings] = X[siblings].replace({-1: 0, 1:0, 2:0, 3:0, 4:1, 5:1, 6:1})
print(len(X[siblings]))
print(len(X[race]))


# Make continuous into range columns
# Define the number of bins
num_bins = 5

# Iterate through columns
for column in X.columns:
    if column == loan_schedule or column == race:
        continue
    if X[column].dtype != 'object' and len(X[column].unique()) > num_bins:
        # Determine bin edges
        bins = pd.qcut(X[column], q=num_bins, labels=False, duplicates='drop') # Potentially would want to do normal distruibution istead of equal bins
        X[column] = bins
    X[column] = X[column].astype('category') # Make every column categorical for now

print(X.shape)
# Get categorical columns
cat_cols = X.select_dtypes(include='category').columns
cat_one_hot = pd.get_dummies(X[cat_cols], drop_first=True)
X = X.drop(cat_cols, axis=1)
X = pd.concat([X, cat_one_hot], axis=1)
print(X.shape)

# Add siblings back in
#X[siblings] = sibs



# Split X into test and train dfs
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)



# Output test and train to seperate csvs
X_train.to_csv('SCF_Siblings_train.csv', index=False)
X_test.to_csv('SCF_Siblings_test.csv', index=False)



(18362, 2494)
[ 1  6  3  2 -1  4  5]
18362
18362
(18362, 2494)
(18362, 3531)


In [62]:
print(X.columns)
# Find columsn that include siblings in the name
sibs = [col for col in X.columns if siblings in col]
print(sibs)
sib_col = X[sibs[0]]
print(sib_col.value_counts())
print(sib_col.dtype)

Index(['x3004', 'x6809', 'x7398_1', 'x7398_2', 'x7398_3', 'x7398_4', 'x7578_5',
       'x7019_1', 'x7019_2', 'x7020_2',
       ...
       'y1_3', 'y1_4', 'x42000_1', 'x42000_2', 'x42000_3', 'x42000_4',
       'x42001_1', 'x42001_2', 'x42001_3', 'x42001_4'],
      dtype='object', length=3531)
['x5913_1']
x5913_1
False    14503
True      3859
Name: count, dtype: int64
bool


In [51]:
print(X[siblings].unique())
print(X[siblings].dtype)

print(X[race].unique())
print(X[race].dtype)

# import torch
# my_t = torch.FloatTensor(X[siblings])


[0 1]
int8
[0 1]
int8
